In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/My Drive/salicon'

/content/drive/.shortcut-targets-by-id/1miA8rtqTtMBSOcF7WVQnBjGRbn32yDoB/salicon


In [3]:
!mkdir 'mlnet'

mkdir: cannot create directory ‘mlnet’: File exists


In [4]:
%cd '/content/drive/My Drive/salicon/mlnet'

/content/drive/.shortcut-targets-by-id/1miA8rtqTtMBSOcF7WVQnBjGRbn32yDoB/salicon/mlnet


## 0. Downloading Data 

In [5]:
# ! pip install gdown

In [6]:
# !gdown https://drive.google.com/uc?id=1g8j-hTT-51IG1UFwP0xTGhLdgIUCW5e5&export=download
# ! unzip images.zip
# !gdown https://drive.google.com/a/umn.edu/uc?id=1PnO7szbdub1559LfjYHMy65EDC4VhJC8&export=download
# ! unzip maps.zip  

Manually do this step



1. Train Images -> images
2. Test Images -> test_images
3. Val Images -> val_images
4. Train maps -> train
5. Val maps -> val


#### Sometimes there will be error related to the data, it is because your drive is slow. Reload again.




In [7]:
%ls

0.png   14.png  19.png  23.png  28.png  6.png        images/        test_maps/
10.png  15.png  1.png   24.png  2.png   7.png        images.zip     train/
11.png  16.png  20.png  25.png  3.png   8.png        maps.zip       val/
12.png  17.png  21.png  26.png  4.png   9.png        saved_models/  val_images/
13.png  18.png  22.png  27.png  5.png   CONV_PRIOR/  test_images/


In [5]:
import cv2
import numpy as np 
import os 
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torchvision.models as models
import time
import torchvision.transforms as transforms
import time
import datetime
import matplotlib.pyplot as plt

imgs_train_path = 'images/'
maps_train_path = 'train/'

imgs_val_path = 'val_images/'
maps_val_path = 'val/'

In [6]:

def padding(img, shape_r=480, shape_c=640, channels=3):
    if channels == 1:
        img_padded = np.zeros((shape_r, shape_c), dtype=np.uint8)
    else:
      img_padded = np.zeros((shape_r, shape_c, channels), dtype=np.uint8)

    original_shape = img.shape
    rows_rate = original_shape[0]/shape_r
    cols_rate = original_shape[1]/shape_c

    if rows_rate > cols_rate:
        new_cols = (original_shape[1] * shape_r) // original_shape[0]
        img = cv2.resize(img, (new_cols, shape_r))
        if new_cols > shape_c:
            new_cols = shape_c
        img_padded[:, ((img_padded.shape[1] - new_cols) // 2):((img_padded.shape[1] - new_cols) // 2 + new_cols)] = img
    else:
        new_rows = (original_shape[0] * shape_c) // original_shape[1]
        img = cv2.resize(img, (shape_c, new_rows))
        if new_rows > shape_r:
            new_rows = shape_r
        img_padded[((img_padded.shape[0] - new_rows) // 2):((img_padded.shape[0] - new_rows) // 2 + new_rows), :] = img

    return img_padded


In [7]:


def preprocess_images(paths, shape_r, shape_c):
    ims = np.zeros((len(paths), shape_r, shape_c, 3))

    for i, path in enumerate(paths):
        original_image = cv2.imread(path)
        padded_image = padding(original_image, shape_r, shape_c, 3)
        ims[i] = padded_image.astype('float')
    
    ims = ims[...,::-1]
    ims /= 255.0
    ims = np.rollaxis(ims, 3, 1)  
    return ims


def preprocess_maps(paths, shape_r, shape_c):
    ims = np.zeros((len(paths), 1, shape_r, shape_c))

    for i, path in enumerate(paths):
        original_map = cv2.imread(path, 0)
        padded_map = padding(original_map, shape_r, shape_c, 1)
        ims[i, 0] = padded_map.astype(np.float32)
        ims[i, 0] /= 255.0
        

    return ims

In [8]:
def generator(b_s, phase_gen='train',th = False):
    if phase_gen == 'train':
        images = [imgs_train_path + f for f in os.listdir(imgs_train_path) if f.endswith('.jpg')]
        maps = [maps_train_path + f for f in os.listdir(maps_train_path) if f.endswith('.png')]
        if th == True:
          model.module.network_update(timesteps=100, leak=1.0)
    elif phase_gen == 'val':
        images = [imgs_val_path + f for f in os.listdir(imgs_val_path) if f.endswith('.jpg')]
        maps = [maps_val_path + f for f in os.listdir(maps_val_path) if f.endswith('.png')]
    else:
        raise NotImplementedError

    images.sort()
    maps.sort()
    counter = 0
    
    while True:
        yield preprocess_images(images[counter:counter + b_s], shape_r, shape_c), preprocess_maps(maps[counter:counter + b_s], shape_r_gt, shape_c_gt)
        if counter + b_s >= len(images):
          break
        counter = counter + b_s

In [9]:
class PoissonGenerator(nn.Module):
  def _init_(self):
    super().__init__()
  
  def forward(self,input):
   out = torch.mul(torch.le(torch.rand_like(input), torch.abs(input)*1.0).float(),torch.sign(input))
   return out
	


In [10]:
class STDB(torch.autograd.Function):

	alpha 	= ''
	beta 	= ''
    
	@staticmethod
	def forward(ctx, input, last_spike):
        
		ctx.save_for_backward(last_spike)
		out = torch.zeros_like(input).cuda()
		out[input > 0] = 1.0
		return out

	@staticmethod
	def backward(ctx, grad_output):
	    		
		last_spike, = ctx.saved_tensors
		grad_input = grad_output.clone()
		grad = STDB.alpha * torch.exp(-1*last_spike)**STDB.beta
		return grad*grad_input, None

class LinearSpike(torch.autograd.Function):
    """
    Here we use the piecewise-linear surrogate gradient as was done
    in Bellec et al. (2018).
    """
    gamma = 0.3 # Controls the dampening of the piecewise-linear surrogate gradient

    @staticmethod
    def forward(ctx, input, last_spike):
        
        ctx.save_for_backward(input)
        out = torch.zeros_like(input).cuda()
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        
        input,     = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad       = LinearSpike.gamma*F.threshold(1.0-torch.abs(input), 0, 0)
        return grad*grad_input, None

In [14]:
# test = list(models.vgg16(pretrained=True).features)
# print(test)

In [15]:
# import torch
# import torch.nn as nn
# import torchvision.models as models
# # pool of square window of size=3, stride=2
#m = nn.MaxPool2d(kernel_size=5, stride=1,padding=2, dilation=1, ceil_mode=False)
#m = nn.AvgPool2d(kernel_size=5, stride=1,padding=2)
# pool of non-square window
# #m = nn.AvgPool2d((3, 2), stride=(2, 1))
# input = torch.randn(16, 640, 30, 40)
# output = m(input)
# print(output.shape)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pdb
import math
from collections import OrderedDict
from matplotlib import pyplot as plt
import copy



class VGG_SNN_STDB(nn.Module):
  def __init__(self,activation='Linear',timesteps=100,leak=1.0,default_threshold=1.0,alpha=0.3,beta=0.01,dropout=0.2,kernel_size=3):
    super().__init__()
    if activation == 'Linear':
      self.act_func = LinearSpike.apply
    elif activation == 'STDB':
      self.act_func = STDB.apply
    
    self.timesteps = timesteps
    self.leak = torch.tensor(leak)
    STDB.alpha = alpha
    STDB.beta = beta
    self.dropout = dropout
    self.kernel_size = kernel_size
    self.input_layer 	= PoissonGenerator().cuda()
    self.threshold 		= {}
    self.mem 			= {}
    self.mask 			= {}
    self.spike 			= {}
    self.cfg = [64, 64, 'A', 128, 128, 'A', 256, 256, 256, 'A', 512, 512, 512, 'AR', 512, 512, 512, 'E']
    self.features = self._make_layers()


    self._initialize_weights2()

    for l in range(len(self.features)):
      if isinstance(self.features[l], nn.Conv2d):
        self.threshold[l] = torch.tensor(default_threshold)

  def _initialize_weights2(self):
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        m.weight.data.normal_(0, math.sqrt(2. / n))
        if m.bias is not None:
          m.bias.data.zero_()
  
  def threshold_update(self, scaling_factor=1.0, thresholds=[]):
    self.scaling_factor = scaling_factor

    for pos in range(len(self.features)):
      if isinstance(self.features[pos], nn.Conv2d):
        if thresholds:
          self.threshold[pos] = torch.tensor(thresholds.pop(0)*self.scaling_factor)
  
  def _make_layers(self):
    in_channels = 3

    layers = []

    for x in (self.cfg):
      stride = 1

      if x == 'A':
        
        #layers.pop()
        layers += [nn.AvgPool2d(kernel_size=2, stride=2,padding=0)]
      elif x == 'AR':
        
        #layers.pop()
        layers += [nn.AvgPool2d(kernel_size=5, stride=1, padding=2)]
      elif x == 'E':
    
        #layers.pop()
        layers += [nn.Dropout(self.dropout),nn.Conv2d(512,64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),bias=False),
                nn.Conv2d(64,1,kernel_size=(1, 1), stride=(1, 1) ,padding=(0, 0),bias = False),
                nn.ReLU(inplace=True)]
      else:
       
        layers += [nn.Conv2d(in_channels,x,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),bias=False),
                   nn.ReLU(inplace=True)]
  
        
          
    
        in_channels = x


    features = nn.Sequential(*layers)

    return features

  def network_update(self, timesteps, leak):
    self.timesteps = timesteps
    self.leak = torch.tensor(leak)

  def neuron_init(self, x):
    self.batch_size = x.size(0)
    self.width = x.size(2)
    self.height = x.size(3)
  

    for l in range(len(self.features)):
      if isinstance(self.features[l], nn.Conv2d):
        self.mem[l] = torch.zeros(self.batch_size,self.features[l].out_channels,self.width,self.height)

        #self.spike[l] = torch.zeros(self.batch_size,self.features[l].out_channels,self.width,self.height)

      elif isinstance(self.features[l], nn.Dropout):
        self.mask[l] = self.features[l](torch.ones(self.mem[l-2].shape))

      
      elif isinstance(self.features[l], nn.AvgPool2d):
        self.width = ((self.width +2*self.features[l].padding - self.features[l].kernel_size)//self.features[l].stride) + 1
        self.height = ((self.height +2*self.features[l].padding - self.features[l].kernel_size)//self.features[l].stride) + 1



    self.spike = copy.deepcopy(self.mem)

    for key,values in self.spike.items():
      for value in values:
        value.fill_(-1000)
  
  def forward(self,x,find_max_mem=False,max_mem_layer=0):
    self.neuron_init(x)
    max_mem=0.0

    for t in range(self.timesteps):
      out_prev = self.input_layer(x)
 

      for l in range(len(self.features)):
        if isinstance(self.features[l],(nn.Conv2d)):
          if find_max_mem and l==max_mem_layer:
            if (self.features[l](out_prev)).max()>max_mem:
              max_mem = (self.features[l](out_prev)).max()
              break

          mem_thr = (self.mem[l].cuda()/self.threshold[l].cuda())- 1.0
          out = self.act_func(mem_thr, (t-1-self.spike[l].cuda()))
          rst = self.threshold[l].cuda()*(mem_thr>0).float()
          self.spike[l] = self.spike[l].cuda().masked_fill(out.bool(),t-1)
          self.mem[l] = self.leak*self.mem[l].cuda()+self.features[l](out_prev)-rst
          out_prev = out.clone()
        elif isinstance(self.features[l], nn.AvgPool2d):
          out_prev = self.features[l](out_prev)
        
        elif isinstance(self.features[l], nn.Dropout):
          out_prev = out_prev*self.mask[l].cuda()

      if find_max_mem and max_mem_layer<len(self.features):
        continue
      prev = len(self.features)

      if not find_max_mem:
        self.mem[prev+1] = out_prev
   
    if find_max_mem:
      return max_mem
   
    return self.mem[prev+1]
    
    

    

In [12]:
!pip3 install torchvision

### Model


In [12]:
import torch
import torch.nn as nn
import torchvision.models as models

def find_threshold(batch_size=512, timesteps=2500, architecture='VGG16'):
    
    model.module.network_update(timesteps=timesteps, leak=1.0)

    pos=0
    thresholds=[]
    
    def find(layer, pos):
        max_act=0
        trained = 0
        
        for i,gt_map in generator(batch_size,th=False):
        #for batch_idx, (data, target) in enumerate(loader):
            i,gt_map = torch.tensor(i.copy(),dtype=torch.float),torch.tensor(gt_map,dtype=torch.float)
            data, target = i.cuda(), gt_map.cuda()

            with torch.no_grad():
                model.eval()
                output = model(data, find_max_mem=True, max_mem_layer=layer)
                if output>max_act:
                    max_act = output.item()

                #f.write('\nBatch:{} Current:{:.4f} Max:{:.4f}'.format(batch_idx+1,output.item(),max_act))
                if i.shape[0] ==0:
                    thresholds.append(max_act)
                    pos = pos+1
                    #f.write(' {}'.format(thresholds))
                    model.module.threshold_update(scaling_factor=1.0, thresholds=thresholds[:])
                    break
            trained = trained + i.shape[0]
            if trained % (i.shape[0]) == 0:
              print ("Findthresholds:{}".format(trained))
        return pos

    if architecture.lower().startswith('vgg'):              
        for l in model.module.features.named_children():
            if isinstance(l[1], nn.Conv2d):
                pos = find(int(l[0]), pos)
        


    return thresholds


### Custom Loss function

## With Prior

In [13]:
# Modified MSE Loss Function
class ModMSELoss(torch.nn.Module):
    def __init__(self,shape_r_gt,shape_c_gt):
        super(ModMSELoss, self).__init__()
        self.shape_r_gt = shape_r_gt
        self.shape_c_gt = shape_c_gt
        
    def forward(self, output , label):
        prior_size = prior.shape
        output_max = torch.max(torch.max(output,2)[0],2)[0].unsqueeze(2).unsqueeze(2).expand(output.shape[0],output.shape[1],self.shape_r_gt,self.shape_c_gt)
        #reg = ( 1.0/(prior_size[0]*prior_size[1]) ) * ( 1 - prior)**2
        loss = torch.mean( ((output / output_max) - label)**2 / (1 - label + 0.1) ) # +  torch.sum(reg)
        return loss

## Training Model

In [14]:
shape_r = 240
shape_c = 320
shape_r_gt = 30
shape_c_gt = 40
last_freeze_layer = 23

In [15]:
#model0 = MLNet(prior_size).cuda()
# give the path where model is stored
cur_list = []
model = VGG_SNN_STDB(activation = 'Linear', timesteps=100, leak=1.0, default_threshold=1.0, alpha=0.3, beta=0.01, dropout=0.3, kernel_size=3).cuda()
model = nn.DataParallel(model)
                                 
PATH = 'saved_models/Copy of 2020-12-09 noconcat_noprior.model'   # path of created model
state = torch.load(PATH)
# print("len: ",len(state))
# for key in state:
#   print("state: ", key)
cur_dict = model.state_dict() 
for key in cur_dict:
  cur_list.append(key)
  #print("cur key: ",key)

#print(len(cur_list))

#input("Enter")
count = 0
cur_count = 0
for key in state:
  if count < 34:
    if count%2==0:
      cur_dict[cur_list[cur_count]] = nn.Parameter(state[key].data)
      cur_count +=1

      #f.write('\n Success: Loaded {} from {}'.format(key, path))
      #print("succcess!")
    # else:
    #   print("mismatch")
    #   #f.write('\n Error: Size mismatch, size of loaded model {}, size of current model {}'.format(state['state_dict'][key].shape, model.state_dict()[key].shape))
    count += 1


# for key in cur_dict:
#   print("now cur key: ",key)
# input("enter")



model.load_state_dict(cur_dict)
#input("enter")



<All keys matched successfully>

In [ ]:
if 'thresholds' in state:
  thresholds = state['thresholds']
  f.write('\n Info: Thresholds loaded from trained ANN: {}'.format(thresholds))
  model.module.threshold_update(scaling_factor = scaling_factor, thresholds=thresholds[:])
else:
  thresholds = find_threshold(batch_size=32, timesteps=1000, architecture='VGG16')
  model.module.threshold_update(scaling_factor=0.7,thresholds = thresholds[:])
  temp = {}
  for key, value in state.items():
    temp[key] = value
  temp['thresholds'] = thresholds
  torch.save(thresholds, 'saved_models/thresholds.pt')
  torch.save(temp,PATH)



Findthresholds:32
Findthresholds:64
Findthresholds:96
Findthresholds:128
Findthresholds:160
Findthresholds:192
Findthresholds:224
Findthresholds:256
Findthresholds:288
Findthresholds:320
Findthresholds:352
Findthresholds:384
Findthresholds:416
Findthresholds:448
Findthresholds:480
Findthresholds:512
Findthresholds:544
Findthresholds:576
Findthresholds:608
Findthresholds:640
Findthresholds:672
Findthresholds:704
Findthresholds:736
Findthresholds:768
Findthresholds:800
Findthresholds:832
Findthresholds:864
Findthresholds:896
Findthresholds:928
Findthresholds:960
Findthresholds:992
Findthresholds:1024
Findthresholds:1056
Findthresholds:1088
Findthresholds:1120
Findthresholds:1152
Findthresholds:1184
Findthresholds:1216
Findthresholds:1248
Findthresholds:1280
Findthresholds:1312


In [23]:

criterion = ModMSELoss(shape_r_gt,shape_c_gt).cuda()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3,weight_decay=0.0005,momentum=0.9,nesterov=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4,weight_decay=5e-4)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
loss_history = []
nb_epochs = 10
batch_size = 8

for epoch in range(nb_epochs):
  t1 = time.time()
  image_trained = 0
  for i,gt_map in generator(batch_size,th=True):
      optimizer.zero_grad()
      i,gt_map = torch.tensor(i.copy(),dtype=torch.float),torch.tensor(gt_map,dtype=torch.float)
      for idx,x in enumerate(i):
        i[idx] = normalize(x)   
      i,gt_map = i.cuda(),gt_map.cuda()
      image_trained += batch_size
      out = model.forward(i)
      loss = criterion(out,gt_map)
      loss.backward()
      optimizer.step()
    
      if image_trained % (batch_size*20) == 0:
        print ("Epochs:{} Images:{} Loss:{}".format(epoch,image_trained,loss.item()) )
  t2 = time.time()
  time_per_epoch = (t2 - t1) / 60.0
  print ('Time taken for epoch-{} : {}m'.format(epoch,time_per_epoch))

RuntimeError: ignored

In [ ]:
## savind model

!mkdir saved_models -p
additional_info = 'my_model'
full_path = 'saved_models/' + str(datetime.datetime.now()) + '_' + additional_info + '.model'
torch.save(model.state_dict(), full_path)

## Model Output 

In [ ]:
# give the path where model is stored
PATH = '2020-12-09 noconcat_noprior.model'   # path of created model
model.load_state_dict(torch.load(PATH))

In [ ]:
num = 30
cnt = 0
for i,gt_map in generator(1,phase_gen='val',th=True):
    fig = plt.figure(figsize=(20, 20))
    org_image = i[0].copy()
    org_image = np.rollaxis(org_image, 0, 3) 
    gt_map = torch.tensor(gt_map,dtype=torch.float)
    gt_map = model1.input_layer(gt_map)
    i = torch.tensor(i.copy(),dtype=torch.float)
    for idx,x in enumerate(i):
        i[idx] = normalize(x)
    i = i.cuda()
    ax1 = plt.subplot(3, 3, 1)
    ax1.set_title('original')
    plt.imshow(org_image)
    # ax1.plot(org_image)
    out0 = model0.forward(i)
    out = model1.forward(out0)
    ax1 = plt.subplot(3, 3, 2)
    ax1.set_title('predicted')
    plt.imshow(out[0].squeeze(0).data.cpu().numpy(),cmap='gray')

    ax1 = plt.subplot(3, 3, 3)
    ax1.set_title('gt')
    plt.imshow(gt_map[0][0].data,cmap='gray')
    
    plt.show()
    fig.savefig('%s.png'%cnt)
    cnt += 1
    if cnt > num:
      break

In [ ]:
# #### comparing 2 models  ## No need to run This

# PATH1 = 'saved_models/2019-11-22 07:24:38.815721_my_model.model'
# PATH2 = 'saved_models2/2019-11-21 10_43_21.727494_my_model.model'
# model1 = MLNet(prior_size).cuda()
# model2 = MLNet(prior_size).cuda()

# model1.load_state_dict(torch.load(PATH1))
# model2.load_state_dict(torch.load(PATH2))


# cnt = 0
# for i,gt_map in generator(1,phase_gen='val'):
#     fig = plt.figure(figsize=(20, 20))
#     org_image = i[0].copy()
#     org_image = np.rollaxis(org_image, 0, 3) 
#     i = torch.tensor(i.copy(),dtype=torch.float)
#     for idx,x in enumerate(i):
#         i[idx] = normalize(x)
#     i = i.cuda()
#     ax1 = plt.subplot(4, 4, 1)
#     ax1.set_title('original')
#     plt.imshow(org_image)
    
#     out1 = model1.forward(i)
#     out2 = model2.forward(i)
    
#     ax1 = plt.subplot(4, 4, 2)
#     ax1.set_title('predicted no prior')
#     plt.imshow(out1[0].squeeze(0).data.cpu().numpy(),cmap='gray')
#     ax1 = plt.subplot(4, 4, 3)
#     ax1.set_title('predicted no Prior')
#     plt.imshow(out2[0].squeeze(0).data.cpu().numpy(),cmap='gray')

#     ax1 = plt.subplot(4, 4, 4)
#     ax1.set_title('gt')
#     plt.imshow(gt_map[0][0],cmap='gray')
    
#     plt.show()
#     fig.savefig('%s.png'%visual_cnt)
#     cnt += 1
#     if cnt > num:
#       break


## Run on test set

In [ ]:
# creating dir for storing prediction of test images
! mkdir test_maps

In [ ]:
test_file = [ f for f in os.listdir('test_images/') if f.endswith('.jpg') ]

for i in test_file:
    img = preprocess_images(['test_images/'+i],shape_r,shape_c)
    img = torch.tensor(img.copy(),dtype=torch.float)
    for idx,x in enumerate(img):
        img[idx] = normalize(x)
    img = img.cuda()
    print(i)
    pred = model.forward(img)
    cv2.imwrite('test_maps/' + i[:-3] + 'png',pred[0].squeeze(0).data.cpu().numpy())

# The code below is for the experimental model after removing the prior from the network and changing the loss function

In [ ]:
# for experimental model without the prior
import torch
import torch.nn as nn
import torchvision.models as models


class MLNet(nn.Module):
    
    def __init__(self):
        super(MLNet, self).__init__()
        features = list(models.vgg16(pretrained = True).features)[:-1]
        
        features[23].stride = 1
        features[23].kernel_size = 5
        features[23].padding = 2
                
        self.features = nn.ModuleList(features).eval() 
        self.fddropout = nn.Dropout2d(p=0.5)
        self.int_conv = nn.Conv2d(1280,64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pre_final_conv = nn.Conv2d(64,1,kernel_size=(1, 1), stride=(1, 1) ,padding=(0, 0))

    def forward(self, x):
        results = []
        for ii,model in enumerate(self.features):
            x = model(x)
            if ii in {16,23,29}:
                results.append(x)
        x = torch.cat((results[0],results[1],results[2]),1) 
        x = self.fddropout(x)
        # 64 filters convolution layer
        x = self.int_conv(x)
        # 1*1 convolution layer
        x = self.pre_final_conv(x)
        x = torch.nn.functional.relu(x,inplace=True)
        return x

# Modified MSE Loss Function
class ModMSELoss(torch.nn.Module):
    def __init__(self,shape_r_gt,shape_c_gt):
        super(ModMSELoss, self).__init__()
        self.shape_r_gt = shape_r_gt
        self.shape_c_gt = shape_c_gt
        
    def forward(self, output , label ):
        output_max = torch.max(torch.max(output,2)[0],2)[0].unsqueeze(2).unsqueeze(2).expand(output.shape[0],output.shape[1],self.shape_r_gt,self.shape_c_gt)
        loss = torch.mean( ((output / output_max) - label)**2 / (1 - label + 0.1) )  #+  torch.sum(reg)
        return loss

# Input Images size
shape_r = 240
shape_c = 320
shape_r_gt = 30
shape_c_gt = 40

last_freeze_layer = 23

model = MLNet().cuda()
for i,param in enumerate(model.parameters()):
  if i < last_freeze_layer:
    param.requires_grad = False
   
criterion = ModMSELoss(shape_r_gt,shape_c_gt).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-4)

In [ ]:
import time
import torchvision.transforms as transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

loss_history = []
nb_epochs = 10
batch_size = 16

for epoch in range(nb_epochs):
  t1 = time.time()
  image_trained = 0
  for i,gt_map in generator(batch_size):
      optimizer.zero_grad()
#       print (i.shape)
      i,gt_map = torch.tensor(i.copy(),dtype=torch.float),torch.tensor(gt_map,dtype=torch.float)
      for idx,x in enumerate(i):
        i[idx] = normalize(x)   
      i,gt_map = i.cuda(),gt_map.cuda()
      image_trained += batch_size
      out = model.forward(i)
      loss = criterion(out,gt_map)
      loss.backward()
      optimizer.step()
    
      if image_trained % (batch_size*20) == 0:
        print ("Epochs:{} Images:{} Loss:{}".format(epoch,image_trained,loss.item()) )
  t2 = time.time()
  time_per_epoch = (t2 - t1) / 60.0
  print ('Time taken for epoch-{} : {}m'.format(epoch,time_per_epoch))

In [ ]:
import time
import datetime

# saving model weight
additional_info = 'my_model'
full_path = 'saved_models/' + str(datetime.datetime.now()) + '_' + additional_info + '.model'
torch.save(model.state_dict(), full_path)

In [ ]:
PATH = # path of the saved model
model.load_state_dict(torch.load(PATH))

Now test on some random images from the dataset

In [ ]:

import matplotlib.pyplot as plt
no_visual = 15
visual_cnt = 0
for i,gt_map in generator(1,phase_gen='val'):
    fig = plt.figure(figsize=(20, 20))
    org_image = i[0].copy()
    org_image = np.rollaxis(org_image, 0, 3) 
    i = torch.tensor(i.copy(),dtype=torch.float)
    for idx,x in enumerate(i):
        i[idx] = normalize(x)
    i = i.cuda()
    ax1 = plt.subplot(3, 3, 1)
    ax1.set_title('original')
    plt.imshow(org_image)
    # ax1.plot(org_image)
    out = model.forward(i)
    
    ax1 = plt.subplot(3, 3, 2)
    ax1.set_title('predicted')
    plt.imshow(out[0].squeeze(0).data.cpu().numpy(),cmap='gray')
    
    ax1 = plt.subplot(3, 3, 4)
    ax1.set_title('gt')
    plt.imshow(gt_map[0][0],cmap='gray')
    
    plt.show()
    visual_cnt += 1
    if visual_cnt > no_visual:
      break

## Below code is to run model on video. You have to run this on your pc to get the results as colab don't support this

In [ ]:
full_path = 'saved_models/' + '2019-11-21 10_43_21.727494_my_model' + '.model'
prior_size = (int(shape_r_gt / 10) , int(shape_c_gt / 10))
model = MLNet(prior_size).cuda()
model.load_state_dict(torch.load(full_path))


# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")


def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)


# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
    ret, img2 = cap.read()
    img = img2
    if ret == True:
    
        img = preprocess_images(img,shape_r,shape_c)
        img = torch.tensor(img.copy(),dtype=torch.float)
        for idx,x in enumerate(img):
            img[idx] = normalize(x)
        img = img.cuda()
        
        pred = model.forward(img)
        #    cv2.imwrite('test_maps/' + i[:-3] + 'png',pred[0].squeeze(0).data.cpu().numpy())
            # Display the resulting frame
        img = pred[0].squeeze(0).data.cpu().numpy()
        img = rescale_frame(img, percent=400)
        img = cv2.resize(img, (img.shape[1]*4, img.shape[0]*4))
        cv2.imshow('Frame', img)
        cv2.imshow('frame2', img2)
            # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
			      break
        
  # Break the loop
    else: 
        break
  
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()